# Analisis de variables numericas parte 1

En este segundo notebook, nos centraremos en un analisis bivariado de las variables numericas, comparando el comportamiento de cada variable dentro de cada cluster con su valor promedio en el conjunto total de datos.

Este proceso es fundamental para identificar las características que definen a cada segmento de clientes. A traves de este analisis, buscamos descubrir los factores clave de segmentacion que nos permitan diferenciar y comprender a fondo cada uno de los clusteres.

In [16]:
# Importamos las librerías necesarias
import pandas as pd
from scipy.stats import mannwhitneyu, levene

df_customer = pd.read_csv("dataset/segmented_customers.csv")
df_customer.head()

,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,Review Rating,Subscription Status,Payment Method,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Preferred Payment Method,Frequency of Purchases,Cluster
0,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Credit Card,Express,Yes,Yes,14,Venmo,Fortnightly,2
1,19,Male,Sweater,Clothing,64,Maine,L,Maroon,Winter,3.1,Yes,Bank Transfer,Express,Yes,Yes,2,Cash,Fortnightly,1
2,50,Male,Jeans,Clothing,73,Massachusetts,S,Maroon,Spring,3.1,Yes,Cash,Free Shipping,Yes,Yes,23,Credit Card,Weekly,2
3,21,Male,Sandals,Footwear,90,Rhode Island,M,Maroon,Spring,3.5,Yes,PayPal,Next Day Air,Yes,Yes,49,PayPal,Weekly,2
4,45,Male,Blouse,Clothing,49,Oregon,M,Turquoise,Spring,2.7,Yes,Cash,Free Shipping,Yes,Yes,31,PayPal,Annually,2


### Preparando los datos: Un enfoque estadístico

A continuación, he preparado un código que implementa 2 funciones que están diseñadas para ayudarme a identificar las variables numericas más relevantes y significativas para describir los clústeres.

Mi motor de análisis se basa en dos pasos clave para asegurar que solo las características más importantes se incluyan en el informe final:

1. Filtro de Relevancia de la Media: Mi primera función actúa como un filtro inicial. Rápidamente descarta las variables cuyas medias en un clúster no son significativamente diferentes a la media general del dataset. Esto me ayuda a ahorrar tiempo de procesamiento y a enfocar mi análisis en los candidatos más prometedores.

2. Pruebas de Significación Estadística: Las siguientes funciones aplican rigurosas pruebas estadísticas, como la prueba U de Mann-Whitney (para la mediana) y la prueba de Levene (para la desviación estándar). Estas pruebas validan si las diferencias observadas son estadísticamente significativas, confirmando la relevancia de la variable para el clúster.


In [17]:
def analyze_numerical_relevance(df, cluster_col, numerical_col, cluster_id):
    """
    Compara la media, mediana y desviación estándar de una variable numérica
    en un clúster específico con la población total.
    
    Args:
        df (pd.DataFrame): El DataFrame que contiene los datos.
        cluster_col (str): El nombre de la columna del clúster.
        numerical_col (str): El nombre de la variable numérica a analizar.
        cluster_id (int): El ID del clúster de interés.

    Returns:
        dict: Un diccionario con las métricas de comparación.
    """
    # Crea un sub-DataFrame para el clúster seleccionado
    cluster_df = df[df[cluster_col] == cluster_id]
    
    # Maneja valores nulos para evitar errores en los cálculos
    cluster_data = cluster_df[numerical_col].dropna()
    total_data = df[numerical_col].dropna()

    if cluster_data.empty or total_data.empty:
        return None

    # Calcula las métricas de tendencia central y dispersión
    cluster_mean = cluster_data.mean()
    cluster_median = cluster_data.median()
    cluster_std = cluster_data.std()
    
    total_mean = total_data.mean()
    total_median = total_data.median()
    total_std = total_data.std()
    
    # Calcula el 'Índice de Relevancia' basado en la media
    relevance_index = cluster_mean / total_mean if total_mean != 0 else np.inf

    results = {
        'Característica Distintiva': numerical_col,
        'Media en Clúster': cluster_mean,
        'Media en el Total': total_mean,
        'Índice de Relevancia': relevance_index,
        'Mediana en Clúster': cluster_median,
        'Mediana en el Total': total_median,
        'Desviación Estándar en Clúster': cluster_std,
        'Desviación Estándar en el Total': total_std,
    }
    
    return results

In [18]:
def get_distinctive_numerical_summary(df, cluster_col, numerical_cols, cluster_id, relevance_threshold=1.1, pvalue_threshold=0.05):
    """
    Genera un resumen de las características numéricas distintivas para un clúster,
    filtrando por relevancia, significancia estadística de la mediana (Mann-Whitney U)
    y significancia de la dispersión (Test de Levene).
    
    Args:
        df (pd.DataFrame): El DataFrame que contiene los datos.
        cluster_col (str): El nombre de la columna del clúster.
        numerical_cols (list): Lista de nombres de columnas numéricas a analizar.
        cluster_id (int): El ID del clúster de interés.
        relevance_threshold (float): Umbral para considerar una media como "distintiva".
        pvalue_threshold (float): Umbral para considerar una diferencia como
                                  estadísticamente significativa.

    Returns:
        pd.DataFrame: Una tabla resumen con las características distintivas y sus p-valores.
    """
    if cluster_id not in df[cluster_col].unique():
        return f"Error: El Clúster {cluster_id} no existe."
    
    distinctive_features_list = []
    
    # Separa el clúster de interés del resto de la población
    cluster_data = df[df[cluster_col] == cluster_id]
    other_data = df[df[cluster_col] != cluster_id]

    for col in numerical_cols:
        # Paso 1: Obtener las métricas de tendencia central y dispersión
        metrics = analyze_numerical_relevance(df, cluster_col, col, cluster_id)
        
        if metrics is None:
            continue

        # Paso 2: Filtro de Relevancia
        if metrics['Índice de Relevancia'] > relevance_threshold:
            # Paso 3: Validación Estadística de la Mediana (Mann-Whitney U)
            try:
                # Comparamos la mediana del clúster con la del resto
                _, p_value_mannwhitney = mannwhitneyu(
                    cluster_data[col].dropna(), 
                    other_data[col].dropna(), 
                    alternative='two-sided'
                )
            except ValueError:
                p_value_mannwhitney = 1.0 

            # Paso 4: Validación Estadística de la Dispersión (Test de Levene)
            try:
                # Comparamos la varianza del clúster con la del resto
                _, p_value_levene = levene(
                    cluster_data[col].dropna(), 
                    other_data[col].dropna()
                )
            except ValueError:
                p_value_levene = 1.0

            # Solo agregamos la característica si al menos una de las pruebas es significativa
            if p_value_mannwhitney < pvalue_threshold or p_value_levene < pvalue_threshold:
                metrics['P-valor Mann-Whitney'] = p_value_mannwhitney
                metrics['P-valor Levene'] = p_value_levene
                distinctive_features_list.append(metrics)
    
    if distinctive_features_list:
        summary_df = pd.DataFrame(distinctive_features_list)
        # Reordenar las columnas para una mejor visualización
        summary_df = summary_df[[
            'Característica Distintiva', 'Media en Clúster', 'Mediana en Clúster',
            'Desviación Estándar en Clúster', 'Índice de Relevancia', 
            'P-valor Mann-Whitney', 'P-valor Levene'
        ]]
        summary_df = summary_df.sort_values(by='Índice de Relevancia', ascending=False)
        return summary_df
    else:
        return f"No se encontraron características numéricas distintivas para el Clúster {cluster_id}."


In [19]:
# Identifica automáticamente las columnas numéricas
numerical_cols = df_customer.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Define la columna de clústeres
cluster_col = 'Cluster'
if cluster_col in numerical_cols:
    numerical_cols.remove(cluster_col)

In [20]:
# Define el clúster a analizar (puedes cambiar este valor)
cluster_to_analyze = 7

# Llama a la función principal para obtener el resultado
summary_result = get_distinctive_numerical_summary(
    df_customer, 
    cluster_col, 
    numerical_cols, 
    cluster_to_analyze, 
    relevance_threshold=1.1,
    pvalue_threshold=0.05
)

# Imprime el resultado en el notebook
print(f"# Resumen de Características Numéricas para el Clúster {cluster_to_analyze}\n")
if isinstance(summary_result, pd.DataFrame):
    # Redondea los valores a 2 decimales para las medias y 4 para los p-valores
    formatted_df = summary_result.round({
        'Media en Clúster': 2, 
        'Mediana en Clúster': 2,
        'Desviación Estándar en Clúster': 2,
        'Índice de Relevancia': 2,
        'P-valor Mann-Whitney': 4,
        'P-valor Levene': 4,
    })
    
    # Imprime la tabla Markdown directamente en el notebook
    print(formatted_df.to_markdown(index=False))
else:
    print(summary_result)

# Resumen de Características Numéricas para el Clúster 7

No se encontraron características numéricas distintivas para el Clúster 7.
